In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score

%matplotlib inline

from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
from numba import jit

#@jit
def mcc(tp, tn, fp, fn):
    sup = tp * tn - fp * fn
    inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if inf==0:
        return 0
    else:
        return sup / np.sqrt(inf)

#@jit
def eval_mcc(y_true, y_prob, show=False):
    idx = np.argsort(y_prob)
    y_true_sort = y_true[idx]
    n = y_true.shape[0]
    nump = 1.0 * np.sum(y_true) # number of positive
    numn = n - nump # number of negative
    tp = nump
    tn = 0.0
    fp = numn
    fn = 0.0
    best_mcc = 0.0
    best_id = -1
    prev_proba = -1
    best_proba = -1
    mccs = np.zeros(n)
    for i in range(n):
        # all items with idx < i are predicted negative while others are predicted positive
        # only evaluate mcc when probability changes
        proba = y_prob[idx[i]]
        if proba != prev_proba:
            prev_proba = proba
            new_mcc = mcc(tp, tn, fp, fn)
            if new_mcc >= best_mcc:
                best_mcc = new_mcc
                best_id = i
                best_proba = proba
        mccs[i] = new_mcc
        if y_true_sort[i] == 1:
            tp -= 1.0
            fn += 1.0
        else:
            fp -= 1.0
            tn += 1.0
    if show:
        y_pred = (y_prob >= best_proba).astype(int)
        score = matthews_corrcoef(y_true, y_pred)
        print(score, best_mcc)
        plt.plot(mccs)
        return best_proba, best_mcc, y_pred
    else:
        return best_mcc

def mcc_eval(y_prob, dtrain):
    y_true = dtrain.get_label()
    best_mcc = eval_mcc(y_true, y_prob)
    return 'MCC', best_mcc

In [2]:
df_train = pd.read_hdf('feats_selected/train_engineered_933cols.hdf5','table')

y = pd.read_csv('../../input_orig/train_numeric.csv', usecols=['Response'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42, stratify=y.values)

In [3]:
del df_train
import gc
gc.collect()

dtrain = xgb.DMatrix(X_train, label=y_train)

del X_train
del y_train
gc.collect()

dval = xgb.DMatrix(X_test, label=y_test)

del X_test
del y_test
gc.collect()

14

In [5]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.1,
    'max_depth': 12,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 20
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1)

val_pred = bst.predict(dval, ntree_limit=bst.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst.get_score(fmap='', importance_type='gain')))

[0]	train-auc:0.749593	eval-auc:0.68344
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.78459	eval-auc:0.697786
[2]	train-auc:0.798696	eval-auc:0.700566
[3]	train-auc:0.813052	eval-auc:0.707085
[4]	train-auc:0.825381	eval-auc:0.706936
[5]	train-auc:0.836157	eval-auc:0.711261
[6]	train-auc:0.845623	eval-auc:0.711447
[7]	train-auc:0.853586	eval-auc:0.713125
[8]	train-auc:0.860578	eval-auc:0.714493
[9]	train-auc:0.864631	eval-auc:0.713818
[10]	train-auc:0.871463	eval-auc:0.716646
[11]	train-auc:0.878703	eval-auc:0.71725
[12]	train-auc:0.886739	eval-auc:0.722083
[13]	train-auc:0.89494	eval-auc:0.722541
[14]	train-auc:0.899416	eval-auc:0.723121
[15]	train-auc:0.906027	eval-auc:0.72432
[16]	train-auc:0.908331	eval-auc:0.724254
[17]	train-auc:0.912298	eval-auc:0.724137
[18]	train-auc:0.915752	eval-auc:0.724686
[19]	train-auc:0.919225	eval-auc:0.724731
[20]	train-auc:0.922349	eval-auc:0

In [10]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.05,
    'max_depth': 12,
    'min_child_weight': 1,
    'gamma': 10,
    'base_score': prior,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 20
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst2 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1)

val_pred = bst2.predict(dval, ntree_limit=bst2.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst2.get_score(fmap='', importance_type='gain')))

[0]	train-auc:0.734959	eval-auc:0.686321
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.743058	eval-auc:0.689537
[2]	train-auc:0.75891	eval-auc:0.69453
[3]	train-auc:0.773643	eval-auc:0.700404
[4]	train-auc:0.778599	eval-auc:0.701377
[5]	train-auc:0.78503	eval-auc:0.705312
[6]	train-auc:0.79125	eval-auc:0.707245
[7]	train-auc:0.792846	eval-auc:0.707808
[8]	train-auc:0.796458	eval-auc:0.708899
[9]	train-auc:0.798056	eval-auc:0.71111
[10]	train-auc:0.807557	eval-auc:0.715789
[11]	train-auc:0.81304	eval-auc:0.717478
[12]	train-auc:0.819107	eval-auc:0.718257
[13]	train-auc:0.823935	eval-auc:0.719575
[14]	train-auc:0.828221	eval-auc:0.720509
[15]	train-auc:0.832628	eval-auc:0.720908
[16]	train-auc:0.836712	eval-auc:0.721982
[17]	train-auc:0.838425	eval-auc:0.721693
[18]	train-auc:0.841227	eval-auc:0.721775
[19]	train-auc:0.844613	eval-auc:0.722772
[20]	train-auc:0.847371	eval-auc:0.

In [14]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'booster': 'dart',
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.05,
    'max_depth': 12,
    'min_child_weight': 1,
    'gamma': 10,
    'base_score': prior,
    'rate_drop': 0.2,
    'skip_drop': 0.5,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 20
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst2 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1)

val_pred = bst2.predict(dval, ntree_limit=bst2.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst2.get_score(fmap='', importance_type='gain')))

[0]	train-auc:0.734959	eval-auc:0.686321
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.737817	eval-auc:0.689537
[2]	train-auc:0.75577	eval-auc:0.69453
[3]	train-auc:0.743058	eval-auc:0.698339
[4]	train-auc:0.778599	eval-auc:0.696613
[5]	train-auc:0.78503	eval-auc:0.700314
[6]	train-auc:0.784991	eval-auc:0.706388
[7]	train-auc:0.790628	eval-auc:0.707113
[8]	train-auc:0.792573	eval-auc:0.709718
[9]	train-auc:0.794649	eval-auc:0.708958
[10]	train-auc:0.790641	eval-auc:0.710519
[11]	train-auc:0.794658	eval-auc:0.7104
[12]	train-auc:0.806329	eval-auc:0.714667
[13]	train-auc:0.808178	eval-auc:0.716171
[14]	train-auc:0.808481	eval-auc:0.717265
[15]	train-auc:0.809829	eval-auc:0.716704
[16]	train-auc:0.810579	eval-auc:0.717706
[17]	train-auc:0.817238	eval-auc:0.718392
[18]	train-auc:0.818323	eval-auc:0.721169
[19]	train-auc:0.817052	eval-auc:0.719687
[20]	train-auc:0.818208	eval-auc:0